# Experiment Design ex2 - CoE: textual and audio features

## Imports

In [1]:
import csv
import pandas as pd
import glob
from tqdm import tqdm
import time

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_validate

## Load data

In [2]:
def load_audio(trainpath = "./data/CoE_dataset_icpr/Dev_Set/audio_descriptors/*",
               testpath = "./data/CoE_dataset_icpr/Test_Set/audio_descriptors/*",
              trainrefpath = "./data/CoE_dataset_icpr/Dev_Set/CoeDevelopmentTrainingdata.csv",
              testrefpath = "./data/CoE_dataset_icpr/Dev_Set/CoeDevelopmentTestdata.csv"):
    """ loads all audio data and includes the target variable
    kwargs:
        trainpath: path to the folder containing all audio descriptor csv files of the train set
        testpath: path to the folder containing all audio descriptor csv files of the test set
        trainrefpath: path to the csv containing filename, movie name and the target variable for the train set
        testrefpath: path to the csv containing filename, movie name and the target variable for the test set
    """
    train = pd.DataFrame()
    test = pd.DataFrame()
    trainfiles = []
    testfiles = []

    # load training data
    for csvpath in tqdm(glob.glob(trainpath), desc='Loading audio train data'):
        trainfiles.append(csvpath.split('/')[-1].split('.csv')[0])
        tmp = pd.DataFrame(pd.read_csv(csvpath, header=None))
        train = train.append(pd.DataFrame(tmp.values.flatten()).transpose())

    # load test data
    for csvpath in tqdm(glob.glob(testpath), desc='Loading audio test data'):
        testfiles.append(csvpath.split('/')[-1].split('.csv')[0])
        tmp = pd.DataFrame(pd.read_csv(csvpath, header=None))
        test = test.append(pd.DataFrame(tmp.values.flatten()).transpose())

    # add filename and target variable
    train['fname'] = trainfiles
    test['fname'] = testfiles
    train = train.merge(pd.read_csv(trainrefpath)[['file_name', 'goodforairplanes']], left_on='fname', right_on='file_name').drop(columns=['file_name'])
    test = test.merge(pd.read_csv(testrefpath)[['file_name', 'goodforairplanes']], left_on='fname', right_on='file_name').drop(columns=['file_name'])
    
    # set file name as index
    train.set_index(['fname'], inplace=True)
    test.set_index(['fname'], inplace=True)
    
    # replace NAs in audio
    train = train.fillna(0)
    test = test.fillna(0)
    
    return train, test

In [3]:
def load_text():
    train = pd.read_csv("./data/CoE_dataset_icpr/Dev_Set/text_descriptors/tdf_idf_dev.csv")
    test = pd.read_csv("./data/CoE_dataset_icpr/Test_Set/text_descriptors/tdf_idf_test.csv")
    
    return train, test

In [4]:
audio_train, audio_test = load_audio()
text_train, text_test = load_text()

Loading audio test data: 100%|██████████| 223/223 [02:41<00:00,  1.38it/s]


## Learning algorithms

In [5]:
def seperate_tvar(train, test, target_var):
    """ seperate training/test set and the target variable
    kwargs:
        train: dataframe containing dependent and independent variables
        test: dataframe containing dependent and independent variables
        target_var: column name of the target variable as a string
    """
    if target_var in train.columns:
        train_y = train[target_var]
        train_x = train.drop(columns=target_var)
    if target_var in test.columns:
        test_y = test[target_var]
        test_x = test.drop(columns=target_var)
    
    return train_y, train_x, test_y, test_x

### TODO
- [ ] Dimensionsproblem zwischen train/test
- [ ] Output der Ergebnisse & Timings

In [6]:
# dict with classifier name: time spent for fitting and classifier obejcts
classifier = {"KNN": (0, KNeighborsClassifier()),
              "SVM": (0, SVC()),
              "Naive bayes": (0, GaussianNB())}

for c in classifier:
    train_y, train_x, test_y, test_x = seperate_tvar(audio_train, audio_test, 'goodforairplanes')
    
    model = classifier[c][1]
    start_time = time.time()
    model.fit(train_x, train_y)
    classifier[c] = time.time() - start_time
    pred_y = model.predict(test_x)
    print(pred_y)
    print(metrics.precision_recall_fscore_support(test_y, pred_y))

ValueError: query data dimension must match training data dimension